In [ ]:
from pathlib import Path
import sys
import argparse
from copy import deepcopy
import pandas as pd
import numpy as np
from tqdm import tqdm
from ray.util.multiprocessing import Pool

from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.vec_env import VecCheckNan

from flycraft.env import FlyCraftEnv
from flycraft.utils_common.load_config import load_config

PROJECT_ROOT_DIR = Path().absolute().parent
if str(PROJECT_ROOT_DIR.absolute()) not in sys.path:
    sys.path.append(str(PROJECT_ROOT_DIR.absolute()))

print(PROJECT_ROOT_DIR)

from utils_my.models.ppo_with_bc_loss import PPOWithBCLoss
from utils_my.sb3.vec_env_helper import get_vec_env, make_env
from utils_my.smoothness.smoothness_measure import smoothness_measure_by_delta, smoothness_measure_by_fft
from utils_my.sb3.my_wrappers import ScaledActionWrapper, ScaledObservationWrapper

/home/ucav/PythonProjects/fly-craft-examples


In [21]:
env = FlyCraftEnv(
    config_file=PROJECT_ROOT_DIR / "configs" / "env" / "env_config_for_ppo_easy.json",
    custom_config={}
)
scaled_obs_env = ScaledObservationWrapper(env)
scaled_act_env = ScaledActionWrapper(scaled_obs_env)
scaled_act_env.reset(seed=1)

({'observation': array([0.5      , 0.5145179, 0.5      , 0.2      , 0.5      , 0.5      ,
         0.5      , 0.25     ], dtype=float32),
  'desired_goal': array([0.20118216, 0.5500515 , 0.44069326], dtype=float32),
  'achieved_goal': array([0.2, 0.5, 0.5], dtype=float32)},
 {'plane_state': {'lef': 0.0,
   'npos': 0.0,
   'epos': 0.0,
   'h': 5000.0,
   'alpha': 2.613220523094039,
   'beta': 0.0,
   'phi': 0.0,
   'theta': 2.613220523094039,
   'psi': 0.0,
   'p': 0.0,
   'q': 0.0,
   'r': 0.0,
   'v': 200.0,
   'vn': 200.00000000000003,
   've': 0.0,
   'vh': 1.7763568394002505e-15,
   'nx': 0.04559349104912459,
   'ny': 0.0,
   'nz': 0.998960076119232,
   'ele': -1.327651593874395,
   'ail': 0.0,
   'rud': 0.0,
   'thrust': 0.0,
   'lon': 122.425,
   'lat': 31.425,
   'mu': 5.088887490067862e-16,
   'chi': 0.0}})

In [22]:
print(env.task.goal_sampler.use_fixed_goal, env.task.goal_sampler.goal_v, env.task.goal_sampler.goal_mu, env.task.goal_sampler.goal_chi)

False 201.18216247002567 9.009273926518706 -21.350423236821975


In [28]:
scaled_act_env.unwrapped.task.goal_sampler.sample_goal()

{'v': 170.34552406761497, 'mu': -4.753733191163009, 'chi': 15.021880357803155}

In [29]:
scaled_act_env.unwrapped.task.goal_sampler.use_fixed_goal = True
scaled_act_env.unwrapped.task.goal_sampler.goal_v = 200
scaled_act_env.unwrapped.task.goal_sampler.goal_mu = 10
scaled_act_env.unwrapped.task.goal_sampler.goal_chi = 30

In [34]:
env.task.goal_sampler.sample_goal()

{'v': 200, 'mu': 10, 'chi': 30}